#### Библиотеки

In [1]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.io import wavfile
import librosa
from keras.models import Sequential
from keras.layers import Dense, Reshape
from tensorflow.keras.metrics import *
from tqdm import tqdm

# Загружали дата фрейм с аудио и преобразованием аудио

In [2]:
dataframe = pd.DataFrame()

path = './audio/'
path_for_data = './numpy_data/'



for name in sorted(os.listdir(path)):
    new_row = {'audio':name[:-4], 'numpy': np.load(f'./numpy_data/{name[:-3]+"npy"}')}
    dataframe = pd.concat([dataframe, pd.DataFrame([new_row])], ignore_index=True)

pd.Series({'audio':name, 'numpy': str(name[-3] + 'npy')})

audio    сожру.wav
numpy         wnpy
dtype: object

# Преобразовывали предложения в цифры

In [3]:
# Sample text data
texts = ['А Агаю', 'А вот и нет.', 'А ну быстро сюда!', 'А-а-а, свежее мясо!', 'А-а-а', 'Ага, иду.', 'Ага', 'Ах ты мелкий...', 'Бессмертие!', 'Блинк дагер!', 'Будешь красоткой с яблоком во рту!', 'Будто одного меня было мало.', 'В кучу!', 'В самом расцвете сил.', 'Вангвард!', 'Вдохни поглубже!', 'Вот и за мной пришли.', 'Враг ушел с верхней линии!', 'Враг ушел с нижней линии!', 'Враг ушел с центральной линии!', 'Время нарезать!', 'Всё в кишках, красота!', 'Вы поглядите, сколько сочного мяса вокруг!', 'Выпотрошить!', 'Давай к делу!', 'Давно пора!', 'Дай освежую... я быстро!', 'Дай подправлю тебе фигуру!', 'Двойной урон!', 'Дело в шляпе!!', 'Дело в шляпе!', 'Дышите глубже, ребята!', 'Жадина-говядина, я такой.', 'Иллюзия!', 'Какой симпатичный фарш!', 'Ко мне-е-е-е!', 'Кому свежих рёбрышек.', 'Кто ребро потерял. Моё будет.', 'Люблю внезапность не приходится чистить кишки вручную.', 'Люблю сокращать чужие калории.', 'М-м, вот это я понимаю - свежий воздух!', 'М-м-ах-м-м', 'М-м-ах-м', 'М-м-м, свежее мясо!', 'М-м-м, устрой лёгким праздник!', 'М-м-м-м', 'М-м', 'Мешки плоти.', 'Милый запах, чё', 'Моё!', 'Мэ-э-э.', 'На верную смерть - это ко мне!', 'На кого замахиваешься!', 'На кусочки!', 'Не пропадать же добру.', 'Не стану тратить всё и сразу.', 'Не твое!', 'Не тут-то было!', 'Не устоял!', 'Не-а!', 'Не-е-ет!', 'Не-е-ет.', 'Невидимость!', 'О, свежее мясо!', 'од топор-р-р!', 'Ой!', 'Ой-ой-ой.', 'Ой... это я что ли', 'Освежую, шмакодявка.', 'Отличная заточка.', 'Отложу, авось пригодится.', 'Падж заждался !', 'Падж тебя приголубит!', 'Пайп оф Инсайт!', 'Паршивый волшебник!!', 'Паршивый волшебник!', 'Первая кровь! У меня ничего в зубах не застряло', 'Перед смертью не надышишься.', 'Подам тебя с криповым соусом!', 'Познакомимся поближе.', 'Поймал!', 'Получай!', 'Порежьте меня на кусочки, я бы сделал для вас то же самое.', 'Последний кусок - мне.', 'Припасу на потом.', 'Пшёл с дороги!', 'Р-р-расчехляю ножи!', 'Резать, рубить!', 'Рублю с двойным усердием!', 'Руки в крови - это про меня.', 'Самое время!', 'Свежее мясо!!!', 'Свежее мясо!!', 'Свежее мясо!', 'Свежее мясо.', 'Свежее мясо', 'Скоростная мясорубка!', 'Слезь!', 'Сожру с потрохами!', 'сожру', 'Солнце уходит на запад, а я остаюсь!', 'Спасёт от передряги.', 'Сюда, быстро!', 'Так пир или... чума!', 'Так тебе и надо!', 'Тип того', 'Туда так туда', 'Тушку мне!', 'Ты ничего не почувствуешь!', 'Ты сегодня без ужина!', 'Тэ-э.', 'У меня уже топор чешется!', 'Урн оф Шадоувс!', 'Ускорение!', 'Форс стафф!', 'Ха-а-а! Свежее мясо!', 'Ха-а', 'Ха-ха! Свежее мясо!', 'Ха-ха, бессмертие!', 'Ха', 'Харт оф тараск!', 'Хе-хе хе', 'Хе-хе-хе-хе-хе-хе-хе, свежее мясо!!', 'Хе-хе-хе-хе-хе-хе-хе, свежее мясо!', 'Хе-хе-хе-хе. Свежее мясо.', 'Хе-хе-хе-хе. Свежее мясо', 'Хе-хе-хе-хе', 'Хм-м, попробуй жизнь на вкус!', 'Хорошая заточка.', 'Худ оф Дефайнс!', 'Хы-хы-ха-ха, ты погляди, свежее мясо!', 'Хы-хы-хы-хы, свежее мясо!', 'Хы-хы-хы-хы, у тебя рёбра отвалились!', 'Хэх, хе-хе-хе-хе-хе, не твоё!', 'Чёрт, давно пора!', 'Чёрта... с два.', 'Чёрта... с два', 'Чёртовы крипы.', 'Что протухло', 'Что-то я... проголодался.', 'Шивас Гвардс!', 'Э-э-э.', 'Э-э.', 'Эй!', 'Эй, ты.', 'Эй', 'Это кто зашел на ужин', 'Эх.', 'Я ж ещё... не дорезал.', 'Я специально, честно...', 'Я что-то не то съел']

# Text normalization and tokenization

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(texts)


def texts_to_padded(texts):
    normalized_texts = []
    global tokenizer
    for text in texts:
        # Normalize text by converting to lowercase and removing punctuation
        normalized_text = text.lower()
        # Tokenize the normalized text
        tokens = tokenizer.texts_to_sequences([normalized_text])[0]
        normalized_texts.append(tokens)

    # Text encoding
    encoded_texts = normalized_texts

    # Padding
    max_seq_length = max(len(seq) for seq in encoded_texts)
    padded_texts = pad_sequences(encoded_texts, maxlen=12, padding='post')

    # Convert to numpy array
    padded_texts = np.array(padded_texts)
    return padded_texts, encoded_texts

padded_texts, encoded_texts = texts_to_padded(texts)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Print the preprocessed data
#print("Normalized Texts:", normalized_texts)
print("Encoded Texts:\n", encoded_texts)
print("Padded Texts:\n", padded_texts)
print("Vocabulary Size:", vocab_size)

Encoded Texts:
 [[6, 66], [6, 25, 20, 67], [6, 68, 26, 35], [6, 6, 6, 3, 4], [6, 6, 6], [36, 69], [36], [27, 14, 70], [37], [71, 72], [73, 74, 7, 75, 76, 77], [78, 79, 15, 38, 80], [13, 81], [13, 82, 83, 84], [85], [86, 87], [25, 20, 88, 89, 90], [28, 29, 7, 91, 30], [28, 29, 7, 92, 30], [28, 29, 7, 93, 30], [39, 94], [40, 13, 95, 96], [97, 98, 99, 100, 101, 102], [103], [104, 105, 106], [41, 42], [43, 44, 8, 26], [43, 107, 45, 108], [109, 110], [46, 13, 47], [46, 13, 47], [111, 112, 113], [114, 115, 8, 116], [117], [118, 119, 120], [48, 21, 16, 16, 16], [121, 122, 123], [49, 124, 125, 50, 126], [51, 127, 5, 128, 129, 130, 131], [51, 132, 133, 134], [2, 2, 25, 17, 8, 135, 136, 137], [2, 2, 27, 2, 2], [2, 2, 27, 2], [2, 2, 2, 3, 4], [2, 2, 2, 138, 139, 140], [2, 2, 2, 2], [2, 2], [141, 142], [143, 144, 145], [50], [146, 11, 11], [12, 147, 148, 17, 48, 21], [12, 149, 150], [12, 52], [5, 151, 53, 152], [5, 153, 154, 40, 20, 155], [5, 156], [5, 157, 18, 38], [5, 158], [5, 6], [5, 16, 54], 

# Загружаем спектрограмму 

In [4]:
spectrograms = [np.load(f'./numpy_data/{file_name}') for file_name in sorted(os.listdir('./numpy_data'))]

ayst = max([i.shape[1] for i in spectrograms])
def norm_spectr(spectrograms):
    a = np.zeros((80,ayst)) -80
    b = spectrograms
    a[:, :b.shape[1]] = b
    return a
spectrograms = [norm_spectr(i) for i in spectrograms]
spectrograms=np.array(spectrograms)



#### Спектрограмму преобразовываем в звук

In [5]:
path_for_data = './numpy_data/'

def spectrogram_to_audio(  mel_spec_db: np.array
                         , output_path: str = './audio2/'
                         , output_filename: str ='output.wav') -> None:

    mel_spec = librosa.db_to_power(mel_spec_db)

    # Reconstruct audio signal from mel-spectrogram
    audio = librosa.feature.inverse.mel_to_audio(mel_spec, sr=22050, n_fft=1024, hop_length=256)

    # Rescale audio to the range [-1, 1]
    audio /= np.max(np.abs(audio))
 
    output_path = os.path.join(output_path, output_filename)
    wavfile.write(output_path, 22050, audio)

spectrogram_to_audio(spectrograms[2])

## Создаём нейронную сеть

In [6]:
train_data_x, train_data_y = np.array(padded_texts), np.array(spectrograms)
output_shape = spectrograms[0].shape
model = Sequential()
model.add( Dense(30, input_dim=len(padded_texts[0])) )
for i in range(4):
        model.add( Dense(units=20, activation='LeakyReLU') )
model.add( Dense( output_shape[0]*output_shape[1], activation='LeakyReLU' ) )
model.add( Reshape(output_shape)  )
model.compile(loss='mae', optimizer='adam')
model.fit(train_data_x, train_data_y
                        , epochs=200
                        , batch_size=64
                        , verbose=0
                        ) 


In [10]:
len(padded_texts),  len(spectrograms)

(151, 151)

#### Обрабатываем текст

In [7]:
word = ['привет как дела?']

def text_to_pedded_text(texts):
    # Text normalization and tokenization
    normalized_texts = []
    tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts(texts)

    for text in texts:
        # Normalize text by converting to lowercase and removing punctuation
        normalized_text = text.lower()
        # Tokenize the normalized text
        tokens = tokenizer.texts_to_sequences([normalized_text])[0]
        normalized_texts.append(tokens)

    # Text encoding
    encoded_texts = normalized_texts

    # Padding
    max_seq_length = max(len(seq) for seq in encoded_texts)
    padded_texts = pad_sequences(encoded_texts, maxlen=max_seq_length, padding='post')

    # Convert to numpy array
    padded_texts = np.array(padded_texts)

    # Vocabulary size
    vocab_size = len(tokenizer.word_index) + 1
    return padded_texts

texts = text_to_pedded_text(word)
texts

array([[1, 2, 3]])

In [8]:
model.predict(texts)

ValueError: in user code:

    File "c:\Users\MateBook\Documents\pyt\.conda\Lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\MateBook\Documents\pyt\.conda\Lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\MateBook\Documents\pyt\.conda\Lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\MateBook\Documents\pyt\.conda\Lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "c:\Users\MateBook\Documents\pyt\.conda\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\MateBook\Documents\pyt\.conda\Lib\site-packages\keras\engine\input_spec.py", li